# LAND LEASE PRICE RECOMMENDER

### Import dependencies

In [1]:
#import libraries
import pandas as pd 
import numpy as np
import random
import math
#import source data
url_1 = 'https://raw.githubusercontent.com/PhilipHarvey20/Land-Lease-Price-Generator/master/acreage_price_by_state.csv'

DURATION DISCOUNT

- weekly = 15%
- monthly = 20%
- 3 months = 35%
- 6 months = 40%
- annually = 60%

### Definition of Fields

In [2]:
#  BASE_STATE_PRC_HUNTING: avg price per acre by state
#  ACREAGE_PRICE_DAILY: ((log(acreage)) * (BASE_STATE_PRC_HUNTING))/30

### Create df

In [3]:
df = pd.read_csv(url_1)
# rename column
df.rename(columns = {'STATE_PRC_PER_ACRE':'BASE_STATE_PRC_HUNTING'},inplace = True)
# Randomly populate activities, acreage
activities = ['HUNTING','FISHING','CAMPING','HIKING','METAL DETECTING','FARMING','SHED ANTLERS','ARTIFACT HUNTING']
df['ACREAGE'] = np.random.randint(1, 2000, df.shape[0])
df['ACTIVITY_TYPE'] = np.random.choice(list(activities), len(df))
# Calculate Total_Acreage_Price (log(acreage) * (base_state_price)
df['ACREAGE_PRICE_DAILY'] = np.nan
df['LOG_BASE_STATE_PRC_HUNTING'] = np.log2(df.BASE_STATE_PRC_HUNTING)
df.loc[df['ACTIVITY_TYPE']=='HUNTING', ['ACREAGE_PRICE_DAILY']] = df['LOG_BASE_STATE_PRC_HUNTING'] * df['ACREAGE']
df['ACREAGE_PRICE_DAILY'] = df['ACREAGE_PRICE_DAILY'].div(30).round(2)
df['DURATION_days'] = np.random.randint(1, 365, df.shape[0])
df = df[['ACTIVITY_TYPE','STATE','DURATION_days','BASE_STATE_PRC_HUNTING','ACREAGE','ACREAGE_PRICE_DAILY']]

In [4]:
# - weekly = 15%
# - monthly = 20%
# - 3 months = 35%
# - 6 months = 40%
# - annually = 60%

In [5]:
# def duration_discount(x):
#     if 0< x <7:
#         return y
#     elif 7<= x <30:
#         return y * .85
#     elif 30<= x <90:
#         return y * .80
#     elif 90<= x <180:
#         return y * .65
#     elif 180<= x <365:
#         return y * .60
#     elif x >= 365:
#         return y *.40

In [6]:
# df['TOTAL_ACREAGE_PRICE'] = df['ACREAGE_PRICE_DAILY'] * df['DURATION_days']
# df['TOTAL_ACREAGE_PRICE'] = df['DURATION_days'].apply(duration_discount)

In [7]:
# df.loc[(df['DURATION_days']>=7) & (df['DURATION_days']<30), [TOTAL_ACREAGE_PRICE]] = [TOTAL_ACREAGE_PRICE

In [8]:
df[df['ACTIVITY_TYPE']=='HUNTING']

,ACTIVITY_TYPE,STATE,DURATION_days,BASE_STATE_PRC_HUNTING,ACREAGE,ACREAGE_PRICE_DAILY
0,HUNTING,Alabama,178,10.0,1054,116.71
1,HUNTING,Arkansas,342,11.0,1704,196.50
15,HUNTING,New York,251,19.0,1038,146.98
20,HUNTING,South Carolina,206,15.0,1707,222.30
28,HUNTING,California,10,10.0,97,10.74
31,HUNTING,Delaware,110,10.0,1042,115.38
36,HUNTING,Massachusetts,346,10.0,747,82.72


In [9]:
df.head(30)

,ACTIVITY_TYPE,STATE,DURATION_days,BASE_STATE_PRC_HUNTING,ACREAGE,ACREAGE_PRICE_DAILY
0,HUNTING,Alabama,178,10.0,1054,116.71
1,HUNTING,Arkansas,342,11.0,1704,196.50
2,FARMING,Georgia,3,15.0,1168,NaN
3,SHED ANTLERS,Illinois,345,25.0,1851,NaN
4,METAL DETECTING,Indiana,171,19.0,1855,NaN
5,ARTIFACT HUNTING,Iowa,111,35.0,1441,NaN
6,HIKING,Kansas,46,9.0,1905,NaN
7,METAL DETECTING,Kentucky,354,14.0,473,NaN
8,SHED ANTLERS,Louisiana,89,9.0,1867,NaN
9,METAL DETECTING,Maryland,77,17.0,1892,NaN
